<a href="https://colab.research.google.com/github/uu-sml/wasp-assigninmen-af-classification/blob/main/assignment_ecg_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WASP Course: Artificial Intelligence and Machine Learning

Lecturer: Dave Zachariah

Assignment responsible: Philipp Pilar, Jingwei Hu, Paul Häusner

# Student and Group Information

Fill this out for the submission of the assignment (you submit this notebook with your solution)

- **Student names:** <font color='red'>Konstantinos Ioannis Sotiropoulos Pesiridis</font>

- **Team ID:** <font color='red'>7N4T9Z3PQ5B6R8F2X1YW</font>

Make sure that the team id is the same as the one with which you submit your model predictions (see coding task 7) such that we can check your performance.

---
# Module 3 - Assignment Overview: ECG classification

The [electrocardiogram (ECG)](https://www.mayoclinic.org/tests-procedures/ekg/about/pac-20384983) records the electrical signals in the heart. It is a common  test used to quickly detect heart problems and to monitor the heart's health. 
In this assignment you will implement and evaluate a model to classify whether the person has [atrial fibrillation (AF)](https://www.mayoclinic.org/diseases-conditions/atrial-fibrillation/symptoms-causes/syc-20350624.) or not based on measurements from the ECG exam. 


**Submission:** You submit the deliverables (see below) at https://canvas.kth.se/courses/48799/assignments

**Due Date:** August 23, 2024.

---
## Basic Tasks
Your task is to implement a classification model, train this model on training data, and evaluate its performance on validation data. We provide skeleton code for the implementation of a simple convolution neural network model.

The steps required to implement this model are presented as numbered tasks below. In total there are seven (7) coding tasks and five (5) explanation tasks. 

## Competitive setting

You have to compute the predictions for the test data (you do not have the labels for it) and submit your predictions to be evaluated to a leaderboard. These predictions will be scored and your submission will be ranked according to the F1 score and compared with your colleagues. In the end a winning team will be determined.

### Deliverables
There are two deliverables:
1. You have to submit this Jupyter notebook on the course web-page (Canvas) together with your code and explanations (where asked for it) that describe your implementation and your experimental results. The notebook should run as a standalone in google colab.
2. You have to have at least **three (3)** submissions (for instructions on how to submit, see coding task 7) where you try to improve the model architecture, the training procedure or the problem formulation. In the submission of this notebook you have to provide a short explanation of what changed between each submission and justify why you decided to make these changes.

### Grading
To pass the assignment, you must submit a complete and working implementation of a model and a well-motivated description and evaluation of it. Your model should reach an Area under the ROC curve (AUROC) on the test data of at least 0.97 and an Average Precision (AP) score of 0.95. Note that the leaderboard to is sorted by F1 score and not AUROC, hence you would want to balance all three metrics.

### GPU Acceleration
To be able to use the GPUs provided by colab in order to speed up your computations, you want to check that the `Hardware accelerator` is set to `GPU` under `Runtime > change runtime type`. Note that notebooks run by connecting to virtual machines that have maximum lifetimes that can be as much as 12 hours. Notebooks will also disconnect from VMs when left idle for too long. 

In [ ]:
import os

# helper function
def exists(path):
    val = os.path.exists(path)
    if val:
        print(f'{path} already exits. Using cached. Delete it manually to recieve it again!')
    return val

# clone requirements.txt if not yet available
if not exists('requirements.txt'):
    !git clone https://gist.github.com/dgedon/8a7b91714568dc35d0527233e9ceada4.git req
    !mv req/requirements.txt .
    !yes | rm -r req

In [ ]:
# Install packages (python>=3.9 is required)
!pip install -r requirements.txt

In [ ]:
# install scikit-learn for auroc
!pip install scikit-learn

In [ ]:
# Import
import torch
import torch.nn as nn
import numpy as np
from tqdm.notebook import trange, tqdm
import h5py
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Suppress matplotlib deprecation warnings
import warnings
from matplotlib import MatplotlibDeprecationWarning
warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)

---
## The data set

The dataset is a subset of the [*CODE dataset*](https://scilifelab.figshare.com/articles/dataset/CODE_dataset/15169716): an anotated database of ECGs. The ECG exams were recorded in Brazil by the Telehealth Network of the state Minas Gerais between 2010 and 2016. The dataset and its usage for the development of deep learning methods was described in ["Automatic diagnosis of the 12-lead ECG using a deep neural network"](https://www.nature.com/articles/s41467-020-15432-4).
The full dataset is available for research upon request.


For the training dataset you have labels. 
For the test dataset you only have the ECG exams but no labels. Evaluation is done by submitting to the leaderboard.

Download the dataset from the given dropbox link and unzip the folder containing the files. The downloaded files are in WFDB format (see [here](https://www.physionet.org/content/wfdb-python/3.4.1/) for details).

In [ ]:
# 1. Download dataset
if not exists('codesubset.tar.gz'):
    !wget https://www.dropbox.com/s/9zkqa5y5jqakdil/codesubset.tar.gz?dl=0 -O codesubset.tar.gz

In [ ]:
# 1. unzip the downloaded data set folder
if not exists('codesubset'):
    !tar -xf codesubset.tar.gz

Note that the extraced folder 'codesubset' contains
1. subfolders with the ECG exam traces. These have to be further preprocessed which we do in the next steps.
2. a csv file which contain the labels and other features for the training data set.


### Preprocessing

Run the cells below to  Clone the GitHub repository which we use for [data preprocessing](https://github.com/antonior92/ecg-preprocessing).

In [ ]:
# 2. clone the code files for data preprocessing
if not exists('ecg-preprocessing'):
    !git clone https://github.com/paulhausner/ecg-preprocessing.git

Let us plot an ECG sample. We can plot ECGs using the `ecg_plot` library for example by using the following code snippet where `ecg_sample` is an array of size `(number of leads * sequence length)`. Now we can view an ECG before preprocessing.

In [ ]:
import ecg_plot
runfile("ecg-preprocessing/read_ecg.py")

PATH_TO_WFDB = 'codesubset/train/TNMG624478'
ecg_sample, sample_rate, _ = read_ecg(PATH_TO_WFDB)

# ECG plot
plt.figure()
lead = ['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
ecg_plot.plot(ecg_sample, sample_rate=sample_rate, style='bw', row_height=8, lead_index=lead, columns=1, title='Sample ECG before pre-processing')
plt.show()


The preprocessing consist of:
- resampling all ECG traces to the sample sampling period (400 Hz). Option: ``--new_freq 400``
- zero padding if necessary such that all ECG have the same number of samples (4096). Option: ``--new_len 4096``.
- removing trends in the ECG signal. Option: ``--remove_baseline``
- remove possible power line noise. Option: ``--powerline 60``

You can run the script bellow to plot the same ECG after the preprocessing.  The script also use the  `ecg_plot` library (as you did above).  You can try also with different command line options to see how the preprocessing affects the signal that will be used by the model.

In [ ]:
%run ecg-preprocessing/plot_from_ecg.py codesubset/train/TNMG624478 --new_freq 400 --new_len 4096 --remove_baseline --powerline 60

Next we perform the preprocessing in all exams and convert them into one single h5 file (see [here](https://www.h5py.org/#:~:text=The%20h5py%20package%20is%20a,they%20were%20real%20NumPy%20arrays.) for details about the format). The resulting h5 files contains the traces as arrays with the shape `(number of traces * sequence length * number of leads)` where sequence length is 4096 and number of leads is 8. 
The files `train.h5` and `test.h5` will be saved inside the folder `codesubset/`.

In [ ]:
# 3. Generate train
if not exists('codesubset/train.h5'):
    !python ecg-preprocessing/generate_h5.py --new_freq 400 --new_len 4096 --remove_baseline --powerline 60 codesubset/train/RECORDS.txt codesubset/train.h5
# 3. Generate test
if not exists('codesubset/test.h5'):
    !python ecg-preprocessing/generate_h5.py --new_freq 400 --new_len 4096 --remove_baseline --powerline 60 codesubset/test/RECORDS.txt codesubset/test.h5

### Coding Task 1: Data Analysis

In [ ]:
# Analysis artifact 1: Accuracy of a random guessing classifier

# Load labels
csv = pd.read_csv("codesubset/train.csv", delimiter=',')
labels = csv["AF"].values

# Calculate number of positive/negative cases
total_samples = len(labels)
positive_samples = np.sum(labels)
negative_samples = total_samples - positive_samples
print(f"Positive samples: {positive_samples}, Negative samples: {negative_samples}")

# Baseline comparison: always guessing the majority class
majority_class_prediction = 1 if positive_samples > negative_samples else 0
majority_accuracy = np.sum(majority_class_prediction == labels) / total_samples
print(f"Majority Classifier Accuracy: {majority_accuracy:.4f}")

# Define a random guessing classifier
def random_guessing_classifier(n_samples, pos_prob):
    return np.random.choice([0, 1], size=n_samples, p=[1 - pos_prob, pos_prob])

# Run the random guessing classifier
pos_prob = positive_samples / total_samples  # Probability of predicting AF
predictions = random_guessing_classifier(total_samples, pos_prob)

# Calculate the expected performance of a random guessing classifier
expected_accuracy = (positive_samples / total_samples) ** 2 + (negative_samples / total_samples) ** 2
print(f"Expected Accuracy of a Random Guessing Classifier: {expected_accuracy:.4f}")

# Evaluate the performance of the random guessing classifier
accuracy = np.sum(predictions == labels) / total_samples
print(f"Random Guessing Classifier Accuracy: {accuracy:.4f}")

In [ ]:
# Analysis artifact 2: An interactive classification game, uncomment last line in cell to play :)
import random
from IPython.display import clear_output

# Load the data
csv = pd.read_csv("codesubset/train.csv", delimiter=',')
f = h5py.File("codesubset/train.h5", "r")
data = f["tracings"]

points = 0
attempts = 0
last_exam_id = -1

# Initialize metrics
TP = 0
FP = 0
FN = 0

# For reproducibility
random.seed(3)

def play_game():
    global points, attempts, last_exam_id, TP, FP, FN
    
    while True:
        
        # Randomly select a row
        row_number = random.randint(0, len(csv) - 1)
        af = csv.iloc[row_number]["AF"]
        exam_id = csv.iloc[row_number]["id_exam"]
        
        # Plot the ECG data
        plt.figure(figsize=(12, 6))
        ecg_plot.plot(data[row_number].T, sample_rate=400, row_height=8, lead_index=lead, columns=1, title=f"Exam {exam_id}")
        plt.show()
        
        # Get user input for the guess
        print(f"Last exam id: {last_exam_id}")
        print(f"Score: {points}/{attempts}")
        print(f"Metrics: TP={TP}, FP={FP}, FN={FN}")
        guess = input("Enter your guess (0 for No AF, 1 for AF, or 2 to quit): ")
        
        # Check if the user wants to quit
        if guess == "2":
            print("Game ended by user.")
            print(f"Final Score: {points}/{attempts}")
            print(f"Final Metrics: TP={TP}, FP={FP}, FN={FN}")
            break
        
        # Convert input to integer and check the answer
        try:
            guess = int(guess)
        except ValueError:
            print("Invalid input, please enter a number (0, 1, or 2).")
            continue
        
        attempts += 1
        
        # Update metrics
        if guess == af:
            points += 1

        if af == 1:
            TP += 1
        
        if guess == 1 and af == 0:  # User predicts AF
            FP += 1
        elif guess == 0 and af == 1:
            FN += 1

        last_exam_id = exam_id
        clear_output(wait=True)

# Start the game, uncomment the line below
# play_game()

### Explanation task 1: Data Analysis

Main Findings from Data Analysis:
- Class Imbalance: The dataset is heavily imbalanced, with 70% of samples being negative for Atrial Fibrillation (AF) and only 30% positive. A classifier that always predicts the majority class (negative) would achieve 70% accuracy.
- Random Guessing: A random guessing classifier, which predicts based on the observed class distribution, would achieve an expected accuracy of about 58%, close to what was observed in practice (58.09%).
- Interactive Classification Experiment: I developed an interactive game to manually classify ECGs as AF or non-AF, using two visual rules: irregular R-R intervals and indistinct P-waves. After educating myself for a couple of hours in yoututebe and over 100 ECGs, I achieved a precision of 56.25%, sensitivity of 58.7%, and an F1-score of 57.4%. Quite disappointing :(

Preprocessing Steps and Their Importance:
- Resampling to 400 Hz: Standardizes the sampling rate across all ECG traces.
- Zero Padding to 4096 Samples: Aligns the length of all ECG signals.
- Removing Baseline Trends: Eliminates slow-moving artifacts from the ECG signal, which do not represent the heart’s electrical activity.
- Removing Power Line Noise: Filters out interference from electrical power sources, which can obscure critical features in the ECG signal, ensuring a cleaner input for the model.


---
## Coding Task 2: My Final Model

In [ ]:
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=16, stride=4, dropout_prob=0.8):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=kernel_size // 2)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=kernel_size // 2 - 1)
        self.bn2 = nn.BatchNorm1d(out_channels)

        # Dropout layers
        self.dropout1 = nn.Dropout(dropout_prob)
        self.dropout2 = nn.Dropout(dropout_prob)

        # Shortcut with Max Pooling followed by 1x1 Conv
        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.MaxPool1d(kernel_size=stride, stride=stride),  # Downsample by a factor of stride (e.g., 4)
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1)  # Match the channel dimension
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout1(out)  # Apply dropout after the first convolution and activation
        out = self.conv2(out)
        out += self.shortcut(x)
        out = F.relu(self.bn2(out))
        out = self.dropout2(out)
        return out


class AFibDetector(nn.Module):
    def __init__(self):
        super(AFibDetector, self).__init__()
        kernel_size = 16
        self.initial_conv = nn.Conv1d(in_channels=8, out_channels=64, kernel_size=kernel_size)
        self.initial_bn = nn.BatchNorm1d(64)

        # Adding 4 Residual Blocks:
        # 1. Increase the number of filters by 64, every 2nd block
        # 2. Subsample by a factor of 4 at every block
        stride = 4
        self.res_block1 = ResidualBlock(in_channels=64, out_channels=64, kernel_size=kernel_size, stride=stride)
        self.res_block2 = ResidualBlock(in_channels=64, out_channels=128, kernel_size=kernel_size, stride=stride)
        self.res_block3 = ResidualBlock(in_channels=128, out_channels=128, kernel_size=kernel_size, stride=stride)
        self.res_block4 = ResidualBlock(in_channels=128, out_channels=256, kernel_size=kernel_size, stride=stride)

        # Calculate the number of flat features after the residual blocks
        dummy_input = torch.zeros(1, 8, 4096)
        dummy_output = self.initial_conv(dummy_input)
        dummy_output = F.relu(self.initial_bn(dummy_output))
        dummy_output = self.res_block1(dummy_output)
        dummy_output = self.res_block2(dummy_output)
        dummy_output = self.res_block3(dummy_output)
        dummy_output = self.res_block4(dummy_output)
        flat_features = dummy_output.numel()

        # Fully Connected Layers
        self.fc1 = nn.Linear(flat_features, 256)
        self.output = nn.Linear(256, 1)

    def forward(self, x):
        x = F.relu(self.initial_bn(self.initial_conv(x)))
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        x = torch.sigmoid(self.output(x))
        return x

### Explanation Task 2: Final Model

- **Lack of Prior Experience with CNNs**:
This is my first time working with Convolutional Neural Networks (CNNs), so I relied heavily on existing literature to guide my model      design choices.

- **Inspiration from Existing Research**: 
The architecture of my model a copy, to the best of my understanding, to the one proposed in the paper titled *"Automatic diagnosis of the 12-lead ECG using a deep neural network"*.

- **Use of Residual Blocks**: 
I chose to implement a 4 Residual Block architecture, as detailed in the paper, to leverage the deep learning benefits of residual networks, such as better gradient flow and the ability to train deeper models. While this architecture is designed to classify ECGs into a variety of conditions, I used it with the understanding that it might be more powerful than necessary for binary classification.

- **Justification for Overkill**:
Despite being potentially overkill for a binary classification task, I opted for this architecture because it has been proven effective in the broader context of ECG classification. I prioritized using a tried-and-tested model over experimenting with simpler architectures due to my limited experience.
I acknowledge that there may be simpler or more efficient models that could achieve similar performance for this specific task.


---
## Train function

The function `train(...)` is called to in every epoch to train the model. The function loads the training data, makes predictions, compares predictions with true labels in the loss function and adapting the model parameters using stochastic gradient descent.

In the code cell below there is the basic structure to load data from the data loader and to log your loss. The arguments of the function are explained by the use in the `main(...)` function below.

If you are unfamiliar with PyTorch training loops, then this official [tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) might help (especially section "4. Train your Network").

### Coding Task 3: Fill training loop

Fill the code cell below such that the model is training when `train(...)` is called.

In [ ]:
def train_loop(epoch, dataloader, model, optimizer, loss_function, device):
    # model to training mode (important to correctly handle dropout or batchnorm layers)
    model.train()
    # allocation
    total_loss = 0  # accumulated loss
    n_entries = 0   # accumulated number of data points
    # progress bar def
    train_pbar = tqdm(dataloader, desc="Training Epoch {epoch:2d}".format(epoch=epoch), leave=True)
    # training loop
    for traces, diagnoses in train_pbar:
        # data to device (CPU or GPU if available)
        traces, diagnoses = traces.to(device), diagnoses.to(device)

        # 1. Zero the gradients
        optimizer.zero_grad()
        # 2. Forward pass: compute the model's predictions for the input batch
        predictions = model(traces)
        # 3. Compute loss: compare the model output with the true labels
        loss = loss_function(predictions, diagnoses)
        # 4. Backward pass: compute the gradient of the loss with respect to model parameters
        loss.backward()
        # 5. Optimization step: update the model's parameters
        optimizer.step()
        
        # Update accumulated values
        total_loss += loss.detach().cpu().numpy()
        n_entries += len(traces)

        # Update progress bar
        train_pbar.set_postfix({'loss': total_loss / n_entries})
    train_pbar.close()
    return total_loss / n_entries

---
## Eval function

The `eval(...)` function is similar to the `train(...)` function but is used to evaluate the model on validation data without adapting the model parameters. You can prohibit computing gradients by using a `with torch.no_grad():` statement.

Currenlty only the loss is logged here. Additionally you have to collect all your predictions and the true values in order to compute more metrics such as AUROC.

### Coding Task 4: Fill evaluation loop
Fill the code cell below such we obtain model predictions to evaluate the validation loss and collect the predictoin in order to compute other validation metrics in the `main(...)` function.

In [ ]:
def eval_loop(epoch, dataloader, model, loss_function, device):
    # model to evaluation mode (important to correctly handle dropout or batchnorm layers)
    model.eval()
    # allocation
    total_loss = 0  # accumulated loss
    n_entries = 0   # accumulated number of data points
    valid_probs = []  # accumulated predicted probabilities
    valid_true = [] # accumulated true labels
    
    # progress bar def
    eval_pbar = tqdm(dataloader, desc="Evaluation Epoch {epoch:2d}".format(epoch=epoch), leave=True)
    # evaluation loop
    for traces_cpu, diagnoses_cpu in eval_pbar:
        # data to device (CPU or GPU if available)
        traces, diagnoses = traces_cpu.to(device), diagnoses_cpu.to(device)

        # Disable gradient computation for efficiency and to prevent changes to the model
        with torch.no_grad():
            # Forward pass: compute the model's predictions for the input batch
            predictions = model(traces)

            # Compute loss: compare the model output with the true labels
            loss = loss_function(predictions, diagnoses)

            # Store probabilities and true labels for later analysis
            valid_probs.append(predictions.cpu().numpy())
            valid_true.append(diagnoses_cpu.numpy())  # Use CPU data directly to avoid unnecessary GPU-to-CPU transfer

        # Update accumulated values
        total_loss += loss.detach().cpu().numpy()
        n_entries += len(traces)

        # Update progress bar
        eval_pbar.set_postfix({'loss': total_loss / n_entries})
    eval_pbar.close()
    return total_loss / n_entries, np.vstack(valid_probs), np.vstack(valid_true)

---
## Run Training

In the code cell below there are some initial (non-optimal!) training hyperparameters. Further, we combine everything from above into training code. That means that we build the dataloaders, define the model/loss/optimizer and then train/validate the model over multiple epochs. Here, we save the model with the lowest validation loss as the best model.

### Coding Task 5: Combine everything to train/validate the model

The following tasks are necessary in the code below
- split the data into training and validation data
- define the loss function
- decide and implement validation metric(s) to evaluate and compare the model on

Optional task:
- include learning rate scheduler
- take specific care about possible data inbalance

### Coding Task 6: Run your model and adapt hyperparameters

After you combined everything in task 5, now you run the code to evaluate the model. Based on the resulting validation metrics you tune
- the training hyperparameters
- the model architecture
- the model hyperparameters.

### Explanation Task 3: Hyperparameter

- **Focus on Key Hyperparameters**:
I concentrated on tuning the learning rate and batch size.

- **Hyperparameter Grid Search**:
I systematically tested the following combinations:
    - **Batch Sizes**: `[8, 16, 32, 64, 128]`
    - **Learning Rates**: `[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01]`

- **Results Documentation**:
  - I recorded the results for each combination in terms of training/validation losses, AUROC, accuracy, and F1 scores over training epochs.
  - The detailed statistics and plots are available in my GitHub repository: [GitHub Repo](https://github.com/kromancer/wasp-assignmen-af-classification), with files named as `"lr_{learning_rate}_bs_{batch_size}.pdf"`.

- **Optimal Hyperparameters**:
The best F1 score was achieved with a learning rate of `1e-4` and a batch size of `8`.

- **Training Duration**:
I used 80 epochs for training. After monitoring the metrics across different epoch counts, I observed that further training beyond 80 epochs did not lead to any noticeable improvement, indicating that the model had likely converged.

- **Motivation**:
The choice of these hyperparameters was driven by empirical evidence from my grid search, focusing on achieving the best possible F1 score while ensuring the model's stability and generalization ability.


In [ ]:
# set seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# choose hyperparameters
learning_rate = 1e-4
weight_decay = 1e-1  
num_epochs = 80
batch_size = 8

In [ ]:
from torch.utils.data import TensorDataset, random_split, DataLoader

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tqdm.write("Use device: {device:}\n".format(device=device))

# =============== Build data loaders ======================================#
tqdm.write("Building data loaders...")

path_to_h5_train, path_to_csv_train, path_to_records = 'codesubset/train.h5', 'codesubset/train.csv', 'codesubset/train/RECORDS.txt'

# load traces
traces = torch.tensor(h5py.File(path_to_h5_train, 'r')['tracings'][()], dtype=torch.float32).transpose(1,2)

# load labels
ids_traces = [int(x.split('TNMG')[1]) for x in list(pd.read_csv(path_to_records, header=None)[0])] # Get order of ids in traces
df = pd.read_csv(path_to_csv_train)
df.set_index('id_exam', inplace=True)
df = df.reindex(ids_traces) # make sure the order is the same
labels = torch.tensor(np.array(df['AF']), dtype=torch.float32).reshape(-1,1)

# load dataset
dataset = TensorDataset(traces, labels)
len_dataset = len(dataset)
n_classes = len(torch.unique(labels))

# split data 
split_ratio = 0.95
split_train = int(len_dataset * split_ratio)
split_valid = len_dataset - split_train
dataset_train, dataset_valid = random_split(dataset, [split_train, split_valid])

# build data loaders
train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)                         
tqdm.write("Done!\n")

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

# =============== Define model ============================================#
tqdm.write("Define model...")
model = AFibDetector()
model.to(device=device)
tqdm.write("Done!\n")

# =============== Define loss function ====================================#
# Since our dataset is imbalanced 7-3 in favor of negatives, let's use a pos weight set to their ratio 7/3
pos_weight = torch.tensor([7/3]).to(device)
loss_function = nn.BCELoss(pos_weight)

# =============== Define optimizer ========================================#
tqdm.write("Define optimiser...")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
tqdm.write("Done!\n")

# =============== Define lr scheduler =====================================#
lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# =============== Train model =============================================#
tqdm.write("Training...")
best_loss = np.Inf

train_loss_all, valid_loss_all = [], []
auroc_all, accuracy_all, f1_all = [], [], []

# loop over epochs
for epoch in trange(1, num_epochs + 1):
    # training loop
    train_loss = train_loop(epoch, train_dataloader, model, optimizer, loss_function, device)
    # validation loop
    valid_loss, y_pred, y_true = eval_loop(epoch, valid_dataloader, model, loss_function, device)

    # collect losses
    train_loss_all.append(train_loss)
    valid_loss_all.append(valid_loss)

    # Use the raw output as it already contains probabilities
    y_pred = torch.tensor(y_pred).numpy()
    auroc = roc_auc_score(y_true, y_pred)
    auroc_all.append(auroc)

    # Convert probabilities to binary predictions
    y_pred_labels = (y_pred >= 0.5).astype(int)
    accuracy = accuracy_score(y_true, y_pred_labels)
    accuracy_all.append(accuracy)
    f1 = f1_score(y_true, y_pred_labels)
    f1_all.append(f1)

    # save best model: here we save the model only for the lowest validation loss
    if valid_loss < best_loss:
        # Save model parameters
        torch.save({'model': model.state_dict()}, 'model.pth') 
        # Update best validation loss
        best_loss = valid_loss
        # statement
        model_save_state = "Best model -> saved"
    else:
        model_save_state = ""

    # Print message
    tqdm.write('Epoch {epoch:2d}: \t'
                'Train Loss {train_loss:.6f} \t'
                'Valid Loss {valid_loss:.6f} \t'
                '{model_save}'
                .format(epoch=epoch,
                        train_loss=train_loss,
                        valid_loss=valid_loss,
                        model_save=model_save_state)
                    )

    # Update learning rate with lr-scheduler
    if lr_scheduler:
        lr_scheduler.step(valid_loss)


# Calculate the best points
best_valid_loss_idx = valid_loss_all.index(min(valid_loss_all))
best_auroc_idx = auroc_all.index(max(auroc_all))
best_accuracy_idx = accuracy_all.index(max(accuracy_all))
best_f1_idx = f1_all.index(max(f1_all))

# Plot Losses
plt.subplot(2, 2, 1)
plt.plot(train_loss_all, label='Train Loss')
plt.plot(valid_loss_all, label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.annotate(f'{valid_loss_all[best_valid_loss_idx]:.6f}',
             xy=(best_valid_loss_idx, valid_loss_all[best_valid_loss_idx]),
             xytext=(best_valid_loss_idx, valid_loss_all[best_valid_loss_idx]),
             arrowprops=dict(facecolor='black', arrowstyle='->'))

# Plot AUROC
plt.subplot(2, 2, 2)
plt.plot(auroc_all, label='Validation AUROC')
plt.title('AUROC over Epochs')
plt.xlabel('Epoch')
plt.ylabel('AUROC')
plt.legend()
plt.annotate(f'{auroc_all[best_auroc_idx]:.4f}',
             xy=(best_auroc_idx, auroc_all[best_auroc_idx]),
             xytext=(best_auroc_idx, auroc_all[best_auroc_idx]),
             arrowprops=dict(facecolor='black', arrowstyle='->'))

# Plot Accuracy
plt.subplot(2, 2, 3)
plt.plot(accuracy_all, label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.annotate(f'{accuracy_all[best_accuracy_idx]:.4f}',
             xy=(best_accuracy_idx, accuracy_all[best_accuracy_idx]),
             xytext=(best_accuracy_idx, accuracy_all[best_accuracy_idx]),
             arrowprops=dict(facecolor='black', arrowstyle='->'))

# Plot F1 Score
plt.subplot(2, 2, 4)
plt.plot(f1_all, label='Validation F1 Score')
plt.title('F1 Score over Epochs')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.annotate(f'{f1_all[best_f1_idx]:.4f}',
             xy=(best_f1_idx, f1_all[best_f1_idx]),
             xytext=(best_f1_idx, f1_all[best_f1_idx]),
             arrowprops=dict(facecolor='black', arrowstyle='->'))

plt.tight_layout()
plt.savefig("validation.pdf")
plt.show()

---
## Model Testing

Since we saved our best model, we can now load the trained model and make predictions on the test data set. We save the predictions in a csv file which will be uploaded as part of the deliverables. Note that we take a `Sigmoid()` function on the model prediction in order to obtain soft predictions (probabilities) instead of hard predictions (0s or 1s).

### Coding Task 7: Make prediction for test data

Here you do not really need to code but you have to:
- replace the baseline model with your model. If you do not use colab then change the path to the model location to load the trained model)
- run the script. The predictions are saved in the variable `soft_pred`.
- upload your predictions to the leaderboard online (see instruction details below). 

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# build the dataloader once and re-use when running the cell below possibly multiple times.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =============== Build data loaders ==========================================#
tqdm.write("Building data loaders...")
path_to_h5_test = 'codesubset/test.h5'
traces = torch.tensor(h5py.File(path_to_h5_test, 'r')['tracings'][()], dtype=torch.float32).transpose(1,2)
dataset = TensorDataset(traces)
len_dataset = len(dataset)
# build data loaders
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
tqdm.write("Done!\n")

In [ ]:
# =============== Define model ================================================#
tqdm.write("Define model...")
model = AFibDetector()

# load stored model parameters
ckpt = torch.load('eclair.pth', map_location=lambda storage, loc: storage)
model.load_state_dict(ckpt['model'])
# put model on device
model.to(device=device)
tqdm.write("Done!\n")

# =============== Evaluate model ==============================================#
model.eval()
# allocation
test_pred = torch.zeros(len_dataset,1)
# progress bar def
test_pbar = tqdm(test_dataloader, desc="Testing")
# evaluation loop
end=0
for traces in test_pbar:
    # data to device
    traces = traces[0].to(device)
    start = end
    with torch.no_grad():
        # Forward pass
        model_output = model(traces)

        # store output
        end = min(start + len(model_output), test_pred.shape[0])
        test_pred[start:end] = torch.nn.Sigmoid()(model_output).detach().cpu()

test_pbar.close()

# =============== Save predictions ============================================#
soft_pred = np.stack((1-test_pred.numpy(), test_pred.numpy()),axis=1).squeeze()

To upload your predictions to the leaderboard, use the following code. There are the following steps to follow:
1. Download the GitHub repository for the leaderboard submission system.
2. Register your team with a **team id** and **password**. The password ensures that only your team can upload to your team id. Do only run the registration once.
3. Upload you predictions as a new submission. There are some things to obey here:
    - For each submission you have to attach a note for you to keep track of the submission in the leaderboard and for us to know which submission you refer to in your explanation. Choose something meaningful such as "submission A" or "model B".
    - You can only get one prediction evaluated per day and you get the score the following day. If you do multiple submissions on the same day, the initial submission will be overwritten and thus only the final submission will be evaluated.
    - Only a maximum of ***FIVE*** submissions will be evaluated. So make them count! (If you update an submission before it is evaluated it doesn't count)
    - The evaluation score is published with you team_id and note at http://hyperion.it.uu.se:5050/leaderboard



In [ ]:
# 1. Download repository for leaderboard submission system
if not exists('leaderboard'):
    !git clone https://gist.github.com/3ff6c4c867331c0bf334301842d753c7.git leaderboard

In [ ]:
# 2. Registration of your team
host = "http://hyperion.it.uu.se:5050/"
runfile("leaderboard/leaderboard_helpers.py")

"""
TASK: Decide for a team_id (max 20 chars) and password. 
Do not change this after you have registered your team
"""
team_id = '' #Fill in a string
password = '' #Fill in a string

# run the registration
r = register_team(team_id, password)
if (r.status_code == 201):
    print("Team registered successfully! Good luck")
elif not (r.status_code == 200):
    raise Exception("You can not change your password once created. If you need help, please contact the teachers")

In [ ]:
# 3. Upload the prediction as submission

# Write a note about the training procedure so you can identify it in the leaderboard. e.g. 5 epochs, or First  (Max 20 characters)
"""
TASK: Add a note for you submission
"""
note = 'dandruff'
team_id = '7N4T9Z3PQ5B6R8F2X1YW'
password = ''

host = "http://hyperion.it.uu.se:5050/"
runfile("leaderboard/leaderboard_helpers.py")

# Submit the predictions to the leaderboard. Note, this also saves your submissions in your colab folder
r = submit(team_id, password, soft_pred.tolist(), note)
if r.status_code == 201:
    print("Submission successful!")
elif r.status_code == 200:
    print("Submission updated!")

### Explanation Task 4: Submissions

1. **Submission "chair"**:
   - **Model**: Used the final model architecture with default hyperparameters.
   - **Bug in Training**: Encountered an issue where I used `BCEWithLogitsLoss`, which re-applied the sigmoid function, despite the model's output already including a sigmoid. The correct loss function should have been `BCELoss`.
   - **Result**: The incorrect loss function led to suboptimal training, impacting accuracy and F1 scores.
2. **Submission "dandruff"**:
   - **Bug Fix**: Corrected the loss function by switching to `BCELoss` to match the sigmoid activation in the model's output layer.
   - **Handling Imbalance**: Noted the low accuracy and F1 score in the previous submission and addressed class imbalance by implementing a weighted loss function, increasing the importance of the minority class (positive cases).
   - **Hyperparameters**: Chose a batch size of 128 and an initial learning rate of 1e-4. These values were selected after some informal experimentation but without a systematic approach.
3. **Submission "eclair"**:
   - **Systematic Hyperparameter Tuning**: Adopted a more structured approach to hyperparameter selection. Specifically, I tested combinations of learning rates (`[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01]`) and batch sizes (`[8, 16, 32, 64, 128]`) to find the optimal settings.
   - **Final Choice**: The selected hyperparameters, lr=0.0001 and bs=8, were based on the combination that provided the best F1 score.

Team id: **<font color='red'>7N4T9Z3PQ5B6R8F2X1YW</font>**

| Submission note | Accuracy | F1    | AUC   | AP    | Submission description                       |
|-----------------|----------|-------|-------|-------|----------------------------------------------|
| chair           | 0.5      | 0.667 | 0.964 | 0.948 | Final model, bug during training             |
| dandruff        | 0.5      | 0.667 | 0.987 | 0.987 | Fixed bug, no hyper tuning, weighted loss fun|
| eclair          | 0.5      | 0.667 | 0.985 | 0.980 | Play with lr and bs, chose lr=1e-4, bs=8     |

### Explanation Task 5: Reflection on Metrics

- **AUC (Area Under the ROC Curve)**:
  - **What it Measures**: AUC measures the ability of the model to distinguish between the positives and negatives across clasification thresholds. It summarizes the trade-off between the true positive rate (sensitivity) and the false positive rate (1-specificity).
  - **Why It's Important**: A high AUC indicates that the model is good at ranking positive examples higher than negative ones, regardless of the specific threshold used for classification. This is useful in situations where the decision threshold may vary depending on the application or when the class distribution is imbalanced.

- **AP (Average Precision)**:
  - **What it Measures**: AP summarizes the precision-recall curve, which focuses on the trade-off between precision and recall for all possible thresholds. It is particularly useful when dealing with imbalanced datasets, as it gives more insight into how well the model handles the minority class.
  - **Why It's Important**: AP is crucial when the positive class is rare, as it evaluates how well the model identifies positives while minimizing false positives. This is especially relevant in applications like medical diagnostics or fraud detection.

- **F1 Score**:
  - **What it Measures**: The F1 score is the harmonic mean of precision and recall, providing a single metric that balances both aspects. It is particularly useful when the costs of false positives and false negatives are different or when you want to avoid extreme values in either precision or recall.
  - **Why It's Important**: F1 score is essential when you need a balance between precision and recall, and neither should be significantly worse than the other. This metric is critical for tasks where both types of errors (false positives and false negatives) have serious consequences.

#### Why Not to Focus on Just One Metric

- **Focusing Only on AUC**:
  - **What Can Happen**: If you only focus on AUC, your model might perform well in ranking instances but might not be well-calibrated for a specific decision threshold. This could lead to suboptimal precision or recall when you actually apply the model to make decisions. For example, a high AUC might coincide with a low F1 score if the model struggles with precision or recall at the chosen threshold (as in my first submission "chair").
  - **Real-World Impact**: In practical scenarios, focusing solely on AUC could result in a model that ranks predictions well but fails to accurately classify instances in a way that meets the specific needs of the application, especially if the positive class is rare.

- **Balancing Metrics**:
  - **Why It's Necessary**: Each metric offers a different perspective on model performance. AUC provides insight into ranking ability, AP focuses on handling the minority class, and F1 score ensures a balance between precision and recall. Focusing on all three metrics ensures that the model is not only good at ranking predictions but also makes reliable classifications that are suitable for the specific context of the problem.
  - **Avoiding Pitfalls**: By considering all these metrics together, you avoid the pitfall of over-optimizing for one aspect of performance at the expense of others, leading to a more robust and well-rounded model.
